## MRI Upload to mri.neurodata.io

This notebook demonstrates how to upload a directory of NIFTI files to a the NeuroData store server.  To use these instructions you need to have created a __"Dataset"__ that describes the shape of the images and a __"Project"__ that names a collections of images.  When you upload images, they will populate __"Channels"__ in the database.  To create a dataset and project, go to mri.neurodata.io/nd/accounts/login/.

### Directory of NIFTI files

In the following text box, give the path to the directory of the MRI files that you want to upload.  The notebook will print a list of files that it has found in that directory.

In [1]:
from ipywidgets import widgets
from IPython.display import display
niidir = widgets.Text()
display("Input directory with .nii files", niidir)

'Input directory with .nii files'

In [2]:
import os
# get the list of files to upload
files = [ os.path.abspath(niidir.value)+'/'+f for f in os.listdir( niidir.value ) ]
print("Files found in directory {}".format(niidir.value))
for f in files:
    if os.path.isfile (f):
        print(f)

Files found in directory /data/scratch
/data/scratch/SWU4_0025805_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/WHS_SD_rat_FA_color_v1.01.nii
/data/scratch/kristina15.sql
/data/scratch/SWU4_0025800_2_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025807_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025801_2_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025808_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/kasthuri11.image_res4.sql
/data/scratch/SWU4_0025800_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025803_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025807_2_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025802_2_DTI_tensors_fa_rgb.nii.gz
/data/scratch/WHS_SD_rat_FA_color_v1.01.nii.gz
/data/scratch/SWU4_0025804_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025804_2_DTI_tensors_fa_rgb.nii.gz
/data/scratch/SWU4_0025806_1_DTI_tensors_fa_rgb.nii.gz
/data/scratch/WHS_SD_rat_T2star_v1.01.nii.gz
/data/scratch/SWU4_0025806_2_DTI_tensors_fa_rgb.nii.gz
/data/scratch/kasthuri11.image_res5.sql
/

### Select files

In the following text box, you will put a python regular expresssion that derives channel names from the listed files.  Use http://regex101.com to help you build an appropriate string.  Paste the file name into test string, select python regular expresssions, and build your expression.  The channel name will be derived from capturing group 1 as shown in the match information on the right side.  For example, I used the regular expression .\*(3d_.\*)_image.\* to capture 3d_int16 and 3d_float as channels from filenames  /.../3d_int16_image.nii.gz and /.../3d_float_image.nii.gz.  You will get to review the channel names and files prior to upload.

In [11]:
inputre = widgets.Text()
display("Regular expresion to extract channel name from each file", inputre)

'Regular expresion to extract channel name from each file'

In [22]:
import re
mritodo={}
for f in files:
    m = re.search(r"{}".format(inputre.value),f)
    if m is not None:
        print ("Upload file {} as channel {}".format(f, m.group(1)))
        mritodo[m.group(1)] = f

Upload file /data/scratch/SWU4_0025805_1_DTI_tensors_fa_rgb.nii.gz as channel 0025805_1
Upload file /data/scratch/SWU4_0025800_2_DTI_tensors_fa_rgb.nii.gz as channel 0025800_2
Upload file /data/scratch/SWU4_0025807_1_DTI_tensors_fa_rgb.nii.gz as channel 0025807_1
Upload file /data/scratch/SWU4_0025801_2_DTI_tensors_fa_rgb.nii.gz as channel 0025801_2
Upload file /data/scratch/SWU4_0025808_1_DTI_tensors_fa_rgb.nii.gz as channel 0025808_1
Upload file /data/scratch/SWU4_0025800_1_DTI_tensors_fa_rgb.nii.gz as channel 0025800_1
Upload file /data/scratch/SWU4_0025803_1_DTI_tensors_fa_rgb.nii.gz as channel 0025803_1
Upload file /data/scratch/SWU4_0025807_2_DTI_tensors_fa_rgb.nii.gz as channel 0025807_2
Upload file /data/scratch/SWU4_0025802_2_DTI_tensors_fa_rgb.nii.gz as channel 0025802_2
Upload file /data/scratch/SWU4_0025804_1_DTI_tensors_fa_rgb.nii.gz as channel 0025804_1
Upload file /data/scratch/SWU4_0025804_2_DTI_tensors_fa_rgb.nii.gz as channel 0025804_2
Upload file /data/scratch/SWU4_0

### Select a Project

Specify the project name on mri.neurodata.io that you already created.

In [21]:
proj = widgets.Text()
display("Project on mri.neurodata.io for mri upload", proj)

'Project on mri.neurodata.io for mri upload'

### Upload the data

Having chosen a project and verified the files and channels that you want to upload, running the following code block will send the data to the server, creating channels for each file.  If your upload fails or is interrupted, you may run this code block multiple times.  It will send the data that has not already been uploaded and omit the data that is already on the server.

In [25]:
import requests
import json

# get the current list of channels
requests.packages.urllib3.disable_warnings()
r = requests.get ( "https://mri.neurodata.io/nd/sd/{}/info/".format(proj.value), verify=False )
infos = json.loads(r.text)
chans = infos["channels"].keys()

for (k,v) in mritodo.items():
    if k in chans:
        print("Skipping already uploaded channel {}".format(k))
        continue
    
    url = 'https://mri.neurodata.io/nd/sd/{}/{}/nii/create/'.format(proj.value, k)

    print("Uploading {} to {}".format(v,url))
    requests.packages.urllib3.disable_warnings()
    response = requests.post(url, open(v, "rb").read(), verify=False)
    print("Status {}. Message {}.".format(response.status_code, response._content ))


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Uploading /data/scratch/SWU4_0025808_1_DTI_tensors_fa_rgb.nii.gz to https://mri.neurodata.io/nd/sd/SWU4_FA/0025808_1/nii/create/
Status 200. Message b'None'.
Uploading /data/scratch/SWU4_0025801_2_DTI_tensors_fa_rgb.nii.gz to https://mri.neurodata.io/nd/sd/SWU4_FA/0025801_2/nii/create/


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Status 200. Message b'None'.
Uploading /data/scratch/SWU4_0025800_2_DTI_tensors_fa_rgb.nii.gz to https://mri.neurodata.io/nd/sd/SWU4_FA/0025800_2/nii/create/


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


...

Status 200. Message b'None'.
Uploading /data/scratch/SWU4_0025802_2_DTI_tensors_fa_rgb.nii.gz to https://mri.neurodata.io/nd/sd/SWU4_FA/0025802_2/nii/create/


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Status 200. Message b'None'.
Uploading /data/scratch/SWU4_0025802_1_DTI_tensors_fa_rgb.nii.gz to https://mri.neurodata.io/nd/sd/SWU4_FA/0025802_1/nii/create/


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


Status 200. Message b'None'.
